# Combining Pre-Built & Custom AI Services

In this notebook, you will integrate with the Computer Vision API and the Text Analytics API to augment the claims processing capabilities. In the end, you will integrate the API calls to the summarizer and classifier services that your deployed and produce a finished claim report that shows all of the processing applied to the claim text and claim image.

## Task 1 - Caption & Tag with the Computer Vision API

In the cell bellow, provided the key to your Computer Vision API and run the cell.

In [1]:
subscription_key = "9fdb8d2ec4e24a699f9a46224a3b0fc9"
assert subscription_key

Construct the endpoint to the Computer Vision API by running the following cell. Notice the last path segment is analyzy, which indicates you will use the analyze feature.

In [2]:
vision_base_url = "https://westus2.api.cognitive.microsoft.com/vision/v1.0/"
vision_analyze_url = vision_base_url + "analyze"

The following cell contains a list of sample images found after performing a simple web search. Feel free to substitute in URL's to the image of your choice. 

In [3]:
fender_bender = "http://ford-life.com/wp-content/uploads/2012/03/Fender-Bender-image.jpg"
damaged_house = "https://c2.staticflickr.com/8/7342/10983313185_0589b74946_z.jpg"
police_car = "https://localtvwnep.files.wordpress.com/2015/11/fender-bender.jpeg?quality=85&strip=all"
car_with_text = "https://static.buildasign.com/cmsimages/bas-vinyl-lettering-splash-01.png?v=4D66584B51394A676D68773D"

From the list of images above, select one and assign it to image_url for further processing:

In [4]:
image_url = police_car

Run the following cell to preview the image you have selected.

In [5]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= image_url, width=400)

The following cell builds the HTTP request to make against the Computer Vision API. 

Run the following cell to retrieve the caption and tags:

In [6]:
import requests
headers  = {'Ocp-Apim-Subscription-Key': subscription_key }
params   = {'visualFeatures': 'Categories,Description,Tags,Color'}
data     = {'url': image_url}
response = requests.post(vision_analyze_url, headers=headers, params=params, json=data)
response.raise_for_status()
analysis = response.json()
analysis

{'categories': [{'name': 'others_', 'score': 0.39453125},
  {'name': 'trans_car', 'score': 0.44140625}],
 'color': {'accentColor': '895D42',
  'dominantColorBackground': 'White',
  'dominantColorForeground': 'White',
  'dominantColors': ['White'],
  'isBwImg': False},
 'description': {'captions': [{'confidence': 0.9485308427051494,
    'text': 'a truck is parked on the side of a road'}],
  'tags': ['outdoor',
   'road',
   'truck',
   'car',
   'parked',
   'street',
   'large',
   'sitting',
   'side',
   'man',
   'bus',
   'white',
   'black',
   'standing',
   'riding',
   'city',
   'driving',
   'red',
   'blue',
   'people',
   'traffic']},
 'metadata': {'format': 'Jpeg', 'height': 1080, 'width': 1920},
 'requestId': '9abe3c92-4c14-42e7-84c3-f87964c8da62',
 'tags': [{'confidence': 0.9950141310691833, 'name': 'outdoor'},
  {'confidence': 0.9936342239379883, 'name': 'road'},
  {'confidence': 0.981715738773346, 'name': 'truck'},
  {'confidence': 0.749627411365509, 'name': 'transpor

As you can see in the above output, the result is a nested document structure. Run the following cells to pull out the caption and top 3 tag results:

In [7]:
caption = analysis["description"]["captions"][0]["text"].capitalize()
caption

'A truck is parked on the side of a road'

In [8]:
topTags = analysis["description"]["tags"][0:3]
topTags

['outdoor', 'road', 'truck']

## Task 2 - Performing OCR

In order to perform OCR with the Computer Vision service, you need to target the OCR endpoint. 

Run the following cell to construct the right URL:

In [9]:
vision_ocr_url = vision_base_url + "ocr"

Next, invoke the OCR endpoint with the following code and examine the result:

In [10]:
headers  = {'Ocp-Apim-Subscription-Key': subscription_key }
params   = {}
data     = {'url': image_url}
response = requests.post(vision_ocr_url, headers=headers, params=params, json=data)
response.raise_for_status()
ocr_analysis = response.json()
ocr_analysis

{'language': 'en',
 'orientation': 'Up',
 'regions': [{'boundingBox': '365,127,937,78',
   'lines': [{'boundingBox': '1028,127,274,49',
     'words': [{'boundingBox': '1028,141,184,35', 'text': 'TAYLOR'},
      {'boundingBox': '1228,127,74,42', 'text': 'Sip'}]},
    {'boundingBox': '365,171,318,34',
     'words': [{'boundingBox': '365,171,318,34', 'text': 'EMERGENCY'}]}]}],
 'textAngle': 0.0}

We have provided the following code for you to extract the text as a flat array from the results.

Run the following cell to extract the text items from the results document:

In [11]:
import itertools
flatten = lambda x: list(itertools.chain.from_iterable(x))
words_list = [[ [w['text'] for w in line['words']]  for line in d['lines']] for d in ocr_analysis['regions']]
words_list = flatten(flatten(words_list))
print(list(words_list))

['TAYLOR', 'Sip', 'EMERGENCY']


## Task 3 - Performing Sentiment Analysis

Sentiment Analysis is performed using the Text Analytics API. 

Update the following cell with the key to your instance of the Text Analytics API and run the cell:

In [12]:
text_analytics_subscription_key = "a967cecfbd6e403d974a75f8a00b8bdb"
assert text_analytics_subscription_key

Update the following cell with the correct base URL for your deployed instance of the Text Analytics API and run the cell:

In [13]:
text_analytics_base_url = "https://westus2.api.cognitive.microsoft.com/text/analytics/v2.0/"
sentiment_api_url = text_analytics_base_url + "sentiment"

The following cell has a set of example claims you can use to test the measurement sentiment. 

Run the cell:

In [14]:
neg_sent = """We are just devastated and emotionally drained. 
The roof was torn off of our car, and to make matters
worse my daughter's favorite teddy bear was impaled on the street lamp."""
pos_sent = """We are just happy the damaage was mininmal and that everyone is safe. 
We are thankful for your support."""
neutral_sent = """I crashed my car."""
long_claim = """
I was driving down El Camino and stopped at a red light.
It was about 3pm in the afternoon.  
The sun was bright and shining just behind the stoplight.
This made it hard to see the lights.
There was a car on my left in the left turn lane.
A few moments later another car, a black sedan pulled up behind me. 
When the left turn light changed green, the black sedan hit me thinking 
that the light had changed for us, but I had not moved because the light 
was still red.
After hitting my car, the black sedan backed up and then sped past me.
I did manage to catch its license plate. 
The license plate of the black sedan was ABC123. 
"""

From the above list of claims, select one and assign its variable to claim_text to be used in the call to the Text Analytics API.

In [15]:
claim_text = long_claim

The API requires you to submit a document of the following form. 

Run the cell to build the request document:

In [16]:
documents = {'documents' : [
    {'id': '1', 'language': 'en', 'text': claim_text}
]}

Now invoke the Text Analytics API and observe the result. 

In [17]:
headers   = {"Ocp-Apim-Subscription-Key": text_analytics_subscription_key}
response  = requests.post(sentiment_api_url, headers=headers, json=documents)
sentiments = response.json()
sentiments

{'documents': [{'id': '1', 'score': 0.5}], 'errors': []}

To parse out the sentiment score from the response, run the following cell:

In [18]:
score = sentiments['documents'][0]['score']
score

0.5

You can provide a human-friendly interpretation on this score by running the following cell:

In [24]:
score_interpretation = "neutral"
if (score < 0.45): 
    score_interpretation = "negative"
elif (score >= 0.55):
    score_interpretation = "positive"
score_interpretation

'neutral'

## Task 4 - Invoking the Azure ML Deployed Services

Run the following cell to define a method that will be used to invoke your classifier and summarizer methods deployed using Azure Machine Learning services:

In [25]:
def invoke_service(ml_service_key, ml_service_scoring_endpoint, ml_service_input):
    headers   = {"Authorization": "Bearer " + ml_service_key}
    response  = requests.post(ml_service_scoring_endpoint, headers=headers, json=ml_service_input)
    result = response.json()
    return result

Configure the classifier invocation with the key and endpoint as appropriate to your deployed instance:

In [28]:
classifier_service_key = "MdhGeZiTO3Abu8nv5n2V1LV8FFchjKgO"
classifier_service_scoring_endpoint = "http://104.46.119.173/api/v1/service/claimclassifier/score"
classifier_service_input = claim_text

Invoke the classifier and observe the result:

In [29]:
classifier_result = invoke_service(classifier_service_key, classifier_service_scoring_endpoint, classifier_service_input)
classifier_result

'1'

Similarly, configure the key and scoring endpoint as appropriate to your summarizer service:

In [30]:
summarizer_service_key = "7xUzUqcfUYmigIBXRjYP6cN0gfP6j9Ng"
summarizer_service_scoring_endpoint = "http://104.46.119.173/api/v1/service/summarizer/score"
summarizer_service_input = claim_text

Invoke the summarizer service and observe the result:

In [31]:
summarizer_result = invoke_service(summarizer_service_key, summarizer_service_scoring_endpoint, summarizer_service_input)
summarizer_result =  summarizer_result[0].replace("\\n", "").strip() if len(summarizer_result) > 0 else "N/A"
summarizer_result

'The sun was bright and shining just behind the stoplight.This made it hard to see the lights.There was a car on my left in the left turn lane.A few moments later another car, a black sedan pulled up behind me.'

## Task 4 -  Summarizing the Results

In this final task, you pull together all of the pieces to display the results of your AI based processing.

Run the following cell and examine the result.

In [32]:
import IPython.display

displayTemplate = """
<div><b>Claim Summary</b></div>
<div>Caption: {}</div>
<div>Tags: {}</div>
<div>Text in Image: {}</div>
<div>Sentiment: {}</div>
<div><img src='{}' width='200px'></div>
<div>Summary: </div>
<div><pre>{} </pre></div>
<div>&nbsp;</div>
<div>Claim:</div>
<div>{}</div>

"""
displayTemplate = displayTemplate.format(caption, ' '.join(topTags), ' '.join(words_list), 
                                         score_interpretation, image_url, summarizer_result, 
                                         claim_text)
IPython.display.display_html(displayTemplate, raw=True)

Claim Summary 
 Caption: A truck is parked on the side of a road 
 Tags: outdoor road truck 
 Text in Image: TAYLOR Sip EMERGENCY 
 Sentiment: neutral 
 
 Summary: 
 The sun was bright and shining just behind the stoplight.This made it hard to see the lights.There was a car on my left in the left turn lane.A few moments later another car, a black sedan pulled up behind me. 
   
 Claim: 
 
I was driving down El Camino and stopped at a red light.
It was about 3pm in the afternoon. 
The sun was bright and shining just behind the stoplight.
This made it hard to see the lights.
There was a car on my left in the left turn lane.
A few moments later another car, a black sedan pulled up behind me. 
When the left turn light changed green, the black sedan hit me thinking 
that the light had changed for us, but I had not moved because the light 
was still red.
After hitting my car, the black sedan backed up and then sped past me.
I did manage to catch its license plate. 
The license plate of the black sedan was ABC123.